<a href="https://colab.research.google.com/github/tonycheung-github/demo_repo/blob/master/step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/experiencor/keras-yolo3.git

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 220, done.
remote: Total 220 (delta 0), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (220/220), 91.27 KiB | 8.30 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [3]:
!pip install -r keras-yolo3/requirements.txt

     |████████████████████████████████| 112kB 4.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 2.4MB 9.1MB/s 
     |████████████████████████████████| 378kB 39.1MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 92kB 11.9MB/s 
     |████████████████████████████████| 20.2MB 7.5MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 1.3MB 50.6MB/s 
     |████████████████████████████████| 276kB 48.2MB/s 
     |████████████████████████████████| 3.8MB 49.1MB/s 
     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 512kB 39.8MB/s 
     |████████████████████████████████| 327kB 50.8MB/s 
  Created wheel for absl-py: filename=absl_py-0.9.0-cp36-none-any.whl size=121933 sha256=963d67ed71a824ca0702049d02eff4ac4d7f925874a83f7e1a4feafa64d66abb
  Stored in direc

In [4]:
!pip install tensorflow-gpu

In [30]:
#!rm math_train.pkl
#!rm math_valid.pkl
#import os
#os.chdir("keras-yolo3")
!python train.py -c config.json

Streaming output truncated to the last 5000 lines.
       0.9697696 , 0.98747915, 0.9246987 , 1.1181083 , 1.0703082 ,
       0.88176143, 1.0281565 , 0.96918833, 1.0324183 , 0.88401383,
       1.1010544 , 0.9974372 , 1.0863944 , 0.88874876, 1.1346    ,
       1.0058303 , 1.0093998 , 0.99356276, 1.2002583 , 0.9872938 ,
       0.9284345 , 1.0137818 , 0.98541874, 0.90748954, 0.98827904,
       0.91072834, 0.94644505, 0.98650074, 0.98883796, 0.9006421 ,
       1.0246339 , 0.96897024, 1.05551   , 1.0408889 , 0.9137429 ,
       1.0266285 , 0.9199952 , 1.057878  , 0.93974465, 0.9598535 ,
       0.97716117, 1.0206044 , 0.85699   , 0.9190582 , 1.0520401 ,
       0.9064683 , 0.93013835, 1.0110482 , 0.9466864 , 0.9530346 ,
       0.9994612 , 0.98363155, 0.9653554 , 1.0741687 , 0.9677934 ,
       1.0746915 , 0.8989531 , 0.89215046, 0.95840305, 1.0119776 ,
       1.0311198 , 1.014353  , 1.091125  , 0.90558994, 0.9921428 ,
       1.0534894 , 1.0350194 , 0.93438816, 0.97896415, 0.89291626,
       1.02

In [28]:
!find / -type f -name "math_weights.h5"

In [17]:
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from skimage.transform import resize
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
import yolo_utils
#from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
#!git clone https://github.com/allanzelener/yad2k.git
from yad2k.yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline


Cloning into 'yad2k'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 2.35 MiB | 27.35 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [18]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    '''
    Function for filtering boxes based on probability and threshold
    '''
    box_scores = box_confidence*box_class_probs
    box_classes = K.argmax(box_scores,-1)
    box_class_scores = K.max(box_scores,-1)
    filtering_mask = box_class_scores>threshold
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
 
    return scores, boxes, classes

In [19]:
def iou(box1, box2):
    '''
    Function to calculate the Intersection of Union (IOU) between two boxes
    '''
    xi1 = max(box1[0],box2[0])
    yi1 = max(box1[1],box2[1])
    xi2 = min(box1[2],box2[2])
    yi2 = min(box1[3],box2[3])
    inter_area = (yi2-yi1)*(xi2-xi1)
    box1_area = (box1[3]-box1[1])*(box1[2]-box1[0])
    box2_area = (box2[3]-box2[1])*(box2[2]-box2[0])
    union_area = box1_area+box2_area-inter_area
    iou = inter_area/union_area
 
    return iou

In [20]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    '''
    Function for suppressing non-maximum boxes
    '''
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes,iou_threshold)
    scores = K.gather(scores,nms_indices)
    boxes = K.gather(boxes,nms_indices)
    classes = K.gather(classes,nms_indices)

    return scores, boxes, classes

In [5]:
yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                   tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                   tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                   tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))

In [10]:
!pwd

/content


In [24]:
!python predict.py -c config.json -i /valid/1.jpg


Using TensorFlow backend.
Traceback (most recent call last):
  File "predict.py", line 132, in <module>
    _main_(args)
  File "predict.py", line 33, in _main_
    infer_model = load_model(config['train']['saved_weights_name'])
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py", line 492, in load_wrapper
    return load_function(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py", line 583, in load_model
    with H5Dict(filepath, mode='r') as h5dict:
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/io_utils.py", line 191, in __init__
    self.data = h5py.File(path, mode=mode)
  File "/usr/local/lib/python3.6/dist-packages/h5py/_hl/files.py", line 408, in __init__
    swmr=swmr)
  File "/usr/local/lib/python3.6/dist-packages/h5py/_hl/files.py", line 173, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5

# New Section